In [2]:
import torch
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification
from torch.utils.data import Dataset
import os
import pandas as pd
import requests
from tqdm.auto import tqdm
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import re
from tensorflow.keras.callbacks  import TensorBoard

/home/nmrbox/0014/vvarenthirarajah/anaconda3/envs/Machine-Learning/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-12-29 11:46:26.738571: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 1.6 MB/s eta 0:00:00:00:0100:02m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 33.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 27.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: torch
    Found existing installation: torch 1.9.1
    Uninstalling torch-1.9.1:
      Successfully uninstalled torch-1.9.1
ERROR: Could not install packages due to an OSError: [Errno 39] Directory not empty: 'lib'



In [11]:
CUDA_LAUNCH_BLOCKING=1.

In [3]:
model_name = 'Rostlab/prot_bert'

In [55]:
# import pandas as pd
# dataset = pd.read_csv("../Fhalab/virus.csv")
# df = dataset[['FASTA_com', 'IC50']].copy()
# df.loc[df['IC50'] <= 10, 'IC50'] = 1
# df.loc[df['IC50'] > 10, 'IC50'] = 0
# df = df.dropna()

In [3]:
# train_df = df.sample(frac=0.8, random_state=0)
# test_df = df.drop(train_df.index)


In [7]:
# train_df.to_csv('train_df.csv')
# test_df.to_csv('test_df.csv')

In [18]:
# data = pd.read_csv("./Data_train_test/test_df.csv",names=['FASTA_com', 'IC50'],skiprows=1)     
# x = list(data['FASTA_com'])
# y = list(data['IC50'])

In [24]:
# y[0]

0.0

In [4]:
class DeepLocDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, split="train", tokenizer_name='Rostlab/prot_bert_bfd', max_length=1024):

        self.datasetFolderPath = 'Data_train_test/'
        self.trainFilePath = os.path.join(self.datasetFolderPath, 'train_df.csv')
        self.testFilePath = os.path.join(self.datasetFolderPath, 'test_df.csv')
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, do_lower_case=False)

        if split=="train":
          self.seqs, self.labels = self.load_dataset(self.trainFilePath)
        else:
          self.seqs, self.labels = self.load_dataset(self.testFilePath)

        self.max_length = max_length

    def load_dataset(self,path):

        df = pd.read_csv(path,names=['FASTA_com', 'IC50'],skiprows=1)     
        self.labels_dic = {0:'Ineff', 1:'Eff'}
        seq = list(df['FASTA_com'])
        df['labels'] = np.where(df['IC50']== 1, 1, 0)
        label = list(df['labels'])

        assert len(seq) == len(label)
        return seq, label

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        seq = " ".join("".join(self.seqs[idx].split()))
        seq = re.sub(r"[UZOB]", "X", seq)

        seq_ids = self.tokenizer(seq, truncation=True, padding='max_length', max_length=self.max_length)

        sample = {key: torch.tensor(val) for key, val in seq_ids.items()}
        sample['labels'] = torch.tensor(self.labels[idx])

        return sample

In [5]:
train_dataset = DeepLocDataset(split="train", tokenizer_name=model_name, max_length=256) # max_length is only capped to speed-up example.
test_dataset = DeepLocDataset(split="test", tokenizer_name=model_name, max_length=256)

In [6]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [7]:
def model_init():
  return AutoModelForSequenceClassification.from_pretrained(model_name)

In [10]:
tensorboard = TensorBoard(log_dir = 'logs/{}'.format(model_name))

https://github.com/agemagician/ProtTrans/blob/master/Fine-Tuning/ProtBert_BFD_FineTuning_MS.ipynb

In [16]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=1,   # batch size per device during training
    per_device_eval_batch_size=10,   # batch size for evaluation
    warmup_steps=1000,               # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,               # How often to print logs
    do_train=True,                   # Perform training
    do_eval=True,                    # Perform evaluation
    evaluation_strategy="epoch",     # evalute after eachh epoch
    gradient_accumulation_steps=64,  # total number of steps before back propagation
    fp16=True,                       # Use mixed precision
    fp16_opt_level="02",             # mixed precision mode
    run_name="ProBert",       # experiment name
    seed=3                           # Seed for experiment reproducibility 3x3
)

trainer = Trainer(
    model_init=model_init,                # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=test_dataset,             # evaluation dataset
    compute_metrics = compute_metrics,    # evaluation metrics
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file config.json from cache at /home/nmrbox/0014/vvarenthirarajah/.cache/huggingface/hub/models--Rostlab--prot_bert/snapshots/3d05bf06e79014892defacad82e0efd06e977ff6/config.json
Model config BertConfig {
  "_name_or_path": "Rostlab/prot_bert",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.0,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 40000,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 30,
  "pad_token_id": 0,
  "position_embedding

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
0,No log,0.657136,0.644809,0.784053,0.644809,1.000000
1,No log,0.605917,0.822404,0.877589,0.789831,0.987288
2,No log,0.553166,0.863388,0.902724,0.834532,0.983051
3,No log,0.492837,0.866120,0.904483,0.837545,0.983051
4,0.613000,0.444367,0.866120,0.904483,0.837545,0.983051
5,0.613000,0.421142,0.860656,0.901354,0.829181,0.987288
6,0.613000,0.388262,0.863388,0.903101,0.832143,0.987288
7,0.613000,0.378942,0.866120,0.904854,0.835125,0.987288
8,0.613000,0.394266,0.846995,0.887550,0.843511,0.936441
9,0.452200,0.338433,0.871585,0.908738,0.838710,0.991525


***** Running Evaluation *****
  Num examples = 366
  Batch size = 10
***** Running Evaluation *****
  Num examples = 366
  Batch size = 10
***** Running Evaluation *****
  Num examples = 366
  Batch size = 10
***** Running Evaluation *****
  Num examples = 366
  Batch size = 10
/home/nmrbox/0014/vvarenthirarajah/anaconda3/envs/Machine-Learning/lib/python3.9/site-packages/transformers/trainer.py:1820: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  nn.utils.clip_grad_norm_(
***** Running Evaluation *****
  Num examples = 366
  Batch size = 10
***** Running Evaluation *****
  Num examples = 366
  Batch size = 10
***** Running Evaluation *****
  Num examples = 366
  Batch size = 10
/home/nmrbox/0014/vvarenthirarajah/anaconda3/env

TrainOutput(global_step=220, training_loss=0.5205667755820534, metrics={'train_runtime': 2346.7694, 'train_samples_per_second': 6.243, 'train_steps_per_second': 0.094, 'total_flos': 8493891870526464.0, 'train_loss': 0.5205667755820534, 'epoch': 9.96})

In [17]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Thu Dec 29 13:01:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:03:00.0 Off |                  N/A |
| N/

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/

In [12]:
!pip uninstall torch-tb-profiler 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
predictions, label_ids, metrics = trainer.predict(test_dataset)


***** Running Prediction *****
  Num examples = 366
  Batch size = 10


In [19]:
idx = 0
sample_ground_truth = test_dataset.labels_dic[int(test_dataset[idx]['labels'])]
sample_predictions =  test_dataset.labels_dic[np.argmax(predictions[idx], axis=0)]
sample_sequence = test_dataset.tokenizer.decode(test_dataset[idx]['input_ids'], skip_special_tokens=True)

In [20]:
print("Sequence: {} \nGround Truth is: {}\nprediction is: {}".format(sample_sequence,
                                                                      sample_ground_truth,
                                                                      sample_predictions))

Sequence: A L A L H F Y P G V Y D D Y G P P I A R G V N A L D K W N 
Ground Truth is: Ineff
prediction is: Ineff


In [21]:
trainer.save_model('model-prot_bert/')

Saving model checkpoint to model-prot_bert/
Configuration saved in model-prot_bert/config.json
Model weights saved in model-prot_bert/pytorch_model.bin
